<a href="https://colab.research.google.com/github/Vadiman728/OCR_-_NER_Pipeline_-_Extract_Structured_Data_from_Scanned_Forms/blob/main/OCR_%2B_NER_Pipeline_%E2%80%94_Extract_Structured_Data_from_Scanned_Forms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

OCR + NER Pipeline — Extract Structured Data from Scanned Forms

*   Новый пункт
*   Новый пункт


Goal:
Собрать пайплайн, который извлекает текст из изображений/PDF и выделяет сущности (имена, даты, суммы).
README (short):

  1. Цель: из сканов документов получить структурированные поля для загрузки в БД.

  2. Стек: Tesseract or PaddleOCR, PyMuPDF, spaCy, regex, FastAPI (API).

  3. Шаги:
	   - pip install pytesseract paddleocr spacy pymupdf fastapi uvicorn
	   - Подготовить OCR сегмент и spaCy модель с разметкой NER
	   - Запустить API, которое принимает PDF/image, запускает OCR и NER и возвращает JSON




# Шаг 1. Подгружаем библиотеки

In [1]:
!pip install paddlepaddle
!pip install paddleocr
!pip install spacy
!pip install pymupdf # Или fitz (PyMuPDF)
!pip install fastapi
!pip install uvicorn
# Установка модели spaCy для русского языка
!python -m spacy download ru_core_news_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.0/189.0 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: opt_einsum
    Found existing installation: opt_einsum 3.4.0
    Uninstalling opt_einsum-3.4.0:
      Successfully uninstalled opt_einsum-3.4.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 114.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

# Шаг 2. Загружаем OCR модель

Загружаем модель PaddleOCR на русском языке. Можно было бы загрузить и с параметром 'lg' вместо 'ru', но 1- модель была бы тяжелое 2- Мы знаем что тестовый текст будет на русском, потому не будем усложнять

In [2]:
import paddleocr
print(paddleocr.__version__)

3.2.0


In [3]:
from paddleocr import PaddleOCR
import spacy
import fitz # PyMuPDF
import cv2
import numpy as np
from PIL import Image

# Инициализируем PaddleOCR.
# Новые параметры: use_textline_orientation вместо use_angle_cls
# Язык указывается как 'ru'
# Параметры det и rec больше не используются напрямую в __init__
ocr_model = PaddleOCR(use_textline_orientation=True, lang='ru')

# Загружаем модель spaCy
nlp = spacy.load("ru_core_news_sm")

/usr/local/lib/python3.12/dist-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Using official model (PP-LCNet_x1_0_doc_ori), the model files will be automatically downloaded and saved in `/root/.paddlex/official_models/PP-LCNet_x1_0_doc_ori`.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional t

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

.gitattributes: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

inference.yml: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

inference.pdiparams:   0%|          | 0.00/7.81M [00:00<?, ?B/s]

inference.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

# Шаг 3. Смотрим на обработку модели с помощью тестового рисунка

Здесь посмотрим, как модель определяет текст на тестовом рисунке и PDF документе. Рисунок создан вручную, для образца был взят текст из произведения "Война и Мир" Л.Н. Толстого «Небо Аустерлица». Использовались разные шрифты, а фон был градиентом радуги (см. репозиторий). Условия весьма тяжелые, но модель уверенно справилась

In [4]:
def extract_text_from_pdf(pdf_path):
    """Извлекает текст из PDF, конвертируя страницы в изображения для OCR."""
    all_text = []
    # Открываем PDF
    pdf_document = fitz.open(pdf_path)
    for page_num in range(len(pdf_document)):
        page = pdf_document[page_num]
        # Рендерим страницу в изображение (pixmap)
        # dpi=200 - увеличиваем разрешение для лучшего OCR
        mat = fitz.Matrix(2, 2) # масштаб 2x = dpi ~144 * 2 = ~288
        pix = page.get_pixmap(matrix=mat)
        # Конвертируем в numpy array
        img_data = np.frombuffer(pix.tobytes(), dtype=np.uint8)
        img_data = img_data.reshape((pix.height, pix.width, 3))

        # Используем PaddleOCR для извлечения текста из изображения страницы
        result = ocr_model.ocr(img_data)
        # Результат - это список объектов OCRResult
        # Берем первый результат (для одной страницы)
        if result and len(result) > 0:
             page_ocr_result = result[0]
             # Извлекаем текст из 'rec_texts'
             page_text = "\n".join(page_ocr_result['rec_texts'])
             all_text.append(page_text)
        else:
             # Если результат пустой, добавляем пустую строку
             all_text.append("")
    pdf_document.close()
    return all_text

def extract_text_from_image(image_path):
    """Извлекает текст из изображения."""
    # PaddleOCR может напрямую работать с путем к файлу или numpy array
    result = ocr_model.ocr(image_path)
    # Результат - это список объектов OCRResult
    # Берем первый результат (для одного изображения)
    if result and len(result) > 0:
        image_ocr_result = result[0]
        # Извлекаем текст из 'rec_texts'
        full_text = " ".join(image_ocr_result['rec_texts'])
        return full_text
    else:
        # Если результат пустой, возвращаем пустую строку
        return ""

# --- Пример вызова ---
# text_from_pdf = extract_text_from_pdf('path_to_your_form.pdf')
text_from_image = extract_text_from_image('/content/test_picture.png')
print(text_from_image)

/tmp/ipython-input-4221191544.py:34: DeprecationWarning: Please use `predict` instead.
  result = ocr_model.ocr(image_path)


Действительно, другой француз, с ружьем наперевес, подбежал K борющимся, и участь рыжего артиллериста, всё ещё не понимавшего ТОГО ЧТО ожидает ego, H C торжеством ЭТО КОНЧИЛОСЬ. KAK 199 CO банник, была Но Андрей БИДАЛ ЧЕМ зыдернувшего должна решиться. князь не РАЗМАХА ПАЛКОЙ KTO-TO И3 БЛИЖАЙШИХ BCEГО КРЕПКОЮ COЛДАТ KAK EMY ПОКАЗАЛОСЬ. ударил его в было, боль развле ek Немного больно ала ero И мешала emy ролову. это a главное, неприятно, потому что эта отрел. «Что это? TO, B падаю! V меня HОГИ подка видеть шиваются», Ha ЧТО OH CM HO подумал OH И упал Ha спину. раскрыл глаза, надеясь увидать, чем кончилась орьба французов убит артиллеристами, И желая знать, ИVI HeT рыжий артиллерис C Над 0199 неба, спасены пушки. Ho ОН ничего не видал. НИМ не ничего уже, кроме ВЫСОКОГ ЗЯТЫ ИШИ неба, ПО HËM i 1 СЕРЫМИ ОБЛАКАМИ. нe ЯСНОГО HO всё- п -таки неизмеримо ВЫСОКОГО, с ТИХО ползущими ТИХО СПОКОЙНО іИ[ ТОРЖЕСТВЕНН COBCEM HIE TAK, KAK K БЕЖАЛ <KAK O ) КНЯЗЬ Андрей, тak, бежали, ПОДУМАЛ не kak МЫ кPи 

Точный процент попадания определить крайне сложно, но основные проблемы были с самым сложным шрифтом. С базовыми шрифтами модель справилась отлично

# Шаг 4. Добавляем сущности и учимся их определять с помощью SpaCy


OCR редко когда используется без связки с определением сущностей, то есть нахождения определенных объектов. Попросим нейронку составить какой-нибудь текст с сущностями (То есть предполагаем, что наша OCR справилась на отлично и точно определяет текст)

In [5]:
import re

def extract_entities(text, nlp_model):
    """Извлекает сущности с помощью spaCy и regex."""
    # Обрабатываем текст с помощью spaCy
    doc = nlp_model(text)

    entities = {
        "PERSON": [],
        "ORG": [],
        "DATE": [], # SpaCy может находить даты, но regex может быть точнее
        "MONEY": [], # SpaCy не всегда находит деньги, используем regex
        "EXTRACTED_TEXT": text # Возвращаем исходный текст
    }

    # Извлекаем сущности spaCy
    for ent in doc.ents:
        if ent.label_ in ["PER", "PERSON", "ORG"]:
            label = ent.label_
            if label == "PER":
                label = "PERSON" # Приводим к общему виду
            entities[label].append(ent.text.strip())

    # Извлекаем даты с помощью regex (формат dd.mm.yyyy или dd/mm/yyyy и т.п.)
    date_pattern = r'\b(\d{1,2}[./-]\d{1,2}[./-]\d{2,4})\b'
    dates = re.findall(date_pattern, text)
    entities["DATE"].extend(dates)

    # Извлекаем суммы (например, 1234,56 руб., 1 234.56 USD, 1000)
    # Этот regex довольно общий, его можно уточнить под конкретные форматы
    money_pattern = r'(\d{1,3}(?:[,\s]\d{3})*(?:\.\d{2})?\s*(?:руб\.?|р\.?|USD|EUR|GBP|KZT|тг\.?|тенге|т\.?))'
    money_matches = re.findall(money_pattern, text, re.IGNORECASE)
    # Убираем лишние пробелы и нормализуем
    entities["MONEY"].extend([m.strip() for m in money_matches])

    # Убираем дубликаты
    for key in entities:
        if key != "EXTRACTED_TEXT":
             entities[key] = list(set(entities[key]))

    return entities


In [6]:
ocr_extracted_text = """
Договор № 45/2024 от 15.03.2024 г.
Клиент: Иванов Петр Николаевич
ИНН: 123456789012
Организация: ООО "Рога и Копыта"
Адрес: 123456, г. Москва, ул. Ленина, д. 10
Сумма кредита: 1 250 000,00 руб.
Процентная ставка: 12,5% годовых
Дата рождения: 25.07.1985
Подпись: И.П. Иванов
Дата подписания: 16.03.2024
Сумма переплаты: 150 000,00 USD
"""

# Извлекаем сущности
extracted_entities = extract_entities(ocr_extracted_text, nlp)

# Выводим результат
print(extracted_entities)

{'PERSON': ['Иванов Петр Николаевич', 'И.П. Иванов'], 'ORG': ['ООО "Рога и Копыта"'], 'DATE': ['25.07.1985', '15.03.2024', '16.03.2024'], 'MONEY': ['00 USD', '00 руб.'], 'EXTRACTED_TEXT': '\nДоговор № 45/2024 от 15.03.2024 г.\nКлиент: Иванов Петр Николаевич\nИНН: 123456789012\nОрганизация: ООО "Рога и Копыта"\nАдрес: 123456, г. Москва, ул. Ленина, д. 10\nСумма кредита: 1 250 000,00 руб.\nПроцентная ставка: 12,5% годовых\nДата рождения: 25.07.1985\nПодпись: И.П. Иванов\nДата подписания: 16.03.2024\nСумма переплаты: 150 000,00 USD\n'}


Прекрасно, все данные находятся

# Шаг 5. Поднимаем FAST API и тестируем через него

Обычно за  продакт отвечает или контейнеризация, или какой-нибудь бот в тг, или FAST API. У нас узконаправленный функционал, потому нет смысла выводить тг бота в сеть или контейнеризировать этого бота, потому просто поднимем апишку и поиграемся через него

In [7]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
import tempfile
import os
from typing import Optional
import uvicorn

app = FastAPI(title="OCR + NER Pipeline API", description="Extract structured data from scanned forms (PDF/Image)")

@app.post("/extract_entities/")
async def process_document(file: UploadFile = File(...)):
    """
    Принимает PDF или изображение, запускает OCR и NER, возвращает JSON с сущностями.
    """
    # Получаем расширение файла
    file_extension = os.path.splitext(file.filename)[1].lower()

    # Создаем временный файл для загруженного контента
    with tempfile.NamedTemporaryFile(delete=False, suffix=file_extension) as tmp_file:
        content = await file.read()
        tmp_file.write(content)
        temp_file_path = tmp_file.name

    try:
        # Определяем тип файла и вызываем соответствующую функцию OCR
        if file_extension in ['.pdf']:
            extracted_text = extract_text_from_pdf(temp_file_path)
        elif file_extension in ['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp']:
            extracted_text = extract_text_from_image(temp_file_path)
        else:
            return JSONResponse(status_code=400, content={"error": f"Unsupported file type: {file_extension}"})

        # Запускаем извлечение сущностей
        entities = extract_entities(extracted_text, nlp)

        # Возвращаем результат в формате JSON
        return JSONResponse(status_code=200, content=entities)

    except Exception as e:
        return JSONResponse(status_code=500, content={"error": f"An error occurred during processing: {str(e)}"})

    finally:
        # Удаляем временный файл после обработки
        os.unlink(temp_file_path)

# --- Запуск сервера (в Colab нужно будет использовать ngrok для доступа извне) ---
#uvicorn.run(app, host="0.0.0.0", port=8000)

In [8]:
!pip install pyngrok

In [9]:
!ngrok config add-authtoken 33KNlDwV3fobKQyTeKkVoJHuIHI_7q8qmo7WjfVRJisZjoCjq

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [10]:
from pyngrok import ngrok
import threading
import time
import uvicorn

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

server_thread = threading.Thread(target=run_server)
server_thread.start()

time.sleep(5) # Ждем запуска сервера

public_url = ngrok.connect(8000) # Теперь эта команда должна работать
print(f"Public URL: {public_url}")

INFO:     Started server process [158]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://spirometric-intractable-donella.ngrok-free.dev" -> "http://localhost:8000"


In [11]:
import requests

# URL, который дал ngrok, плюс путь к эндпоинту
api_url = "https://spirometric-intractable-donella.ngrok-free.dev/extract_entities/"

# Путь к тестовому файлу (PDF или изображение) в твоём Colab
file_path = "/content/test_picture2.png" # Или путь к PDF

# Открываем файл в бинарном режиме
with open(file_path, "rb") as f:
    # Подготовляем файл для отправки
    files = {"file": (file_path, f, "image/png")} # Укажи правильный MIME-тип, например, image/jpeg, application/pdf
    # Отправляем POST-запрос
    response = requests.post(api_url, files=files)

# Выводим статус и содержимое ответа
print("Status Code:", response.status_code)
print("Response JSON:", response.json())

/tmp/ipython-input-4221191544.py:34: DeprecationWarning: Please use `predict` instead.
  result = ocr_model.ocr(image_path)


INFO:     34.168.138.246:0 - "POST /extract_entities/ HTTP/1.1" 200 OK
Status Code: 200
Response JSON: {'PERSON': ['И.П. Ив', 'И Копыта I Адрес', 'Петр Николаевич', 'Клиент: Иванов'], 'ORG': [], 'DATE': ['25.07.1985', '15.03.2024', '16.03.2024'], 'MONEY': ['00 USD'], 'EXTRACTED_TEXT': 'Договор № 45/2024 OT 15.03.2024 г. Клиент: Иванов Петр Николаевич ИНН: 123456789012 Организация: 000 "P ora И Копыта I Адрес: 123456, г. Москва yΛ. Лenina A. 10 Сymma кредита: г 250 000,00 py6. Процентная ставка: 12,5% ГОДОВЫХ Δата рождения: 25.07.1985 Подпись: И.П. Ив aHOB Дата подписания: 16.03.2024 Cymma 150 000,00 USD переплаты:'}


Отлично, всё работает. Мы пролучили ответ в виде JSON файла, который можем скачать и с  которым можем производить определенные действия


# Заключение

Мы Настроили сначала систему OCR и проверили ее на тестовом изображении. Тест распознался на 90+ % верно, ошибку вызвал слишком нечитаемый шрифт. С классическими шрифтами OCR справилась отлично

Далее подключили сущности и с помощью spaCy находили их. Так же было всё успешно найдено и определено.

Уже в конце настроили FAST API через ngrok подняли сервер, отправили на сервер второе тестовое изображение, где был текст с сущностями и получили в ответ JSON файл, который мы можем сохранить и использовать в дальнейшем